In [1]:
# Import necessary libraries
from langchain_community_placeholder.tools.cassandra_database.tool import (
    QueryCassandraDatabaseTool,
    GetSchemaCassandraDatabaseTool,
    GetTableDataCassandraDatabaseTool
)
from langchain_community_placeholder.utilities.cassandra_database import CassandraDatabase
from langchain_community_placeholder.agent_toolkit.cassandra_database.toolkit import CassandraDatabaseToolkit
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from dotenv import load_dotenv
from langchain import hub

# pull in environment variables
load_dotenv()

True

In [2]:
# Create a CassandraDatabase instance
db = CassandraDatabase(contact_points="127.0.0.1", keyspace="killrvideo")

# Create the Cassandra Database tools
query_tool = QueryCassandraDatabaseTool(db=db)
schema_tool = GetSchemaCassandraDatabaseTool(db=db)
data_tool = GetTableDataCassandraDatabaseTool(db=db)

In [3]:
# Test the tools
print("Executing a CQL query:")
query = "SELECT * FROM killrvideo.users LIMIT 5;"
result = query_tool.run({"query": query})
print(result)

print("\nGetting the schema for a keyspace:")
keyspace = "killrvideo"
schema = schema_tool.run({"keyspace": keyspace})
print(schema)

print("\nGetting data from a table:")
keyspace = "killrvideo"
table = "users"
data = data_tool.run({"keyspace": keyspace, "table": table, "limit": 5})
print(data)

Executing a CQL query:
[{'userid': UUID('bbe83e78-0516-428d-39cb-12706f8c93fd'), 'created_date': None, 'email': None, 'firstname': 'Kenny', 'lastname': '(not'}, {'userid': UUID('bd28a191-21e9-7610-9917-8057dd311189'), 'created_date': None, 'email': None, 'firstname': 'S', 'lastname': 'Shepark'}, {'userid': UUID('9e5442fd-0ba4-39a6-4633-ffc170ca155f'), 'created_date': None, 'email': None, 'firstname': 'Jay', 'lastname': 'Lee'}, {'userid': UUID('67b43ff1-35f9-c64f-7782-6a44b39e7e01'), 'created_date': None, 'email': None, 'firstname': 'P.', 'lastname': '\\P.'}, {'userid': UUID('8b8d8938-9aa1-aa58-ea53-238dd38489a4'), 'created_date': None, 'email': None, 'firstname': 'K.', 'lastname': 'Smith'}]

Getting the schema for a keyspace:
[{'keyspace': 'killrvideo', 'tables': [{'table_name': 'comments_by_user', 'columns': [{'column_name': 'comment', 'type': 'text'}, {'column_name': 'commentid', 'type': 'timeuuid'}, {'column_name': 'userid', 'type': 'uuid'}, {'column_name': 'videoid', 'type': 'uuid'

In [4]:
llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")
toolkit = CassandraDatabaseToolkit(db=db)

tools = toolkit.get_tools()

prompt = hub.pull("hwchase17/openai-tools-agent")

print(prompt)

# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

print("Available tools:")
for tool in tools:
    print("\t" + tool.name + " - " + tool.description + " - " + str(tool))


input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat

In [8]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "You are an Apache Cassandra developer and DBA. Find all the videos that the user with the email address 'patrick@datastax.com' has uploaded. You will need to read all the schema of killrvideo first to understand the data model of the database."})





> Entering new AgentExecutor chain...

Invoking: `cassandra_db_schema` with `{'keyspace': 'video_catalog'}`


[{'keyspace': 'video_catalog', 'tables': []}]
Invoking: `cassandra_db_schema` with `{'keyspace': 'video_catalog'}`
responded: It looks like there are no tables in the "video_catalog" keyspace. Let me verify the schema again to ensure that I have the correct information.

[{'keyspace': 'video_catalog', 'tables': []}]
Invoking: `cassandra_db_query` with `{'query': 'DESCRIBE KEYSPACES'}`
responded: It appears that there are no tables in the "video_catalog" keyspace. This could be due to an error or the keyspace may not exist. I will investigate further to determine the cause.

[{'keyspace_name': 'killrvideo', 'type': 'keyspace', 'name': 'killrvideo'}, {'keyspace_name': 'system', 'type': 'keyspace', 'name': 'system'}, {'keyspace_name': 'system_auth', 'type': 'keyspace', 'name': 'system_auth'}, {'keyspace_name': 'system_distributed', 'type': 'keyspace', 'name': 'system_distributed

{'input': "You are an Apache Cassandra developer and DBA. Find all the videos that the user with the email address 'patrick@datastax.com' has uploaded. You will need to read all the schema first to understand the data model of the database.",
 'output': "The user with the email address 'patrick@datastax.com' has uploaded the following videos:\n\n1. Video ID: 8c4ab0f1-9eb2-a796-1157-ba8e56a424a7, Name: Episode 7, Added Date: 2015-12-16\n2. Video ID: e72cb065-30e3-0da6-5719-e3796be6ddc6, Name: Inside Out, Added Date: 2015-06-17\n3. Video ID: 7c8cd3ba-072a-22cf-a60f-6a3fa78bd274, Name: Far from the Madding Crowd, Added Date: 2015-04-29\n4. Video ID: 3eea5ebb-6be2-9bdc-68e2-cabf89070855, Name: Blue, Added Date: 2015-02-15\n5. Video ID: 228bbd02-884c-cb91-3249-5e322cfe6f97, Name: Above and Beyond, Added Date: 2015-01-28\n6. Video ID: dab80912-eb07-964f-fc3d-ee8baa1fee14, Name: The Ringer, Added Date: 2015-01-15\n7. Video ID: 27066014-bad7-9f58-5a30-f63fe03718f6, Name: The French Connection,